We can use the logistic model in estimating the size and the peak time of the coronavirus epidemic in Philippines. Logistic model uses a logistic function.

A logistic function or logistic curve is a common S-shaped curve (sigmoid curve) with equation

$f(x)={\frac {L}{1+e^{-k(x-x_{0})}}}$

where

*e* = the natural logarithm base (also known as Euler's number)

*$x_{0}$* = the x value of the sigmoid's midpoint,

*L* = the curve's maximum value,

*k* = the logistic growth rate or steepness of the curve.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymc3 as pm
from scipy import optimize
from datetime import datetime

import matplotlib.dates as mdates


tab10 = ['#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD', 
         '#8C564B', '#CFECF9', '#7F7F7F', '#BCBD22', '#17BECF']

%matplotlib inline

In [ ]:
#### Get the data
df = pd.read_csv('ph_covid19_daily_data.csv')
df = df.fillna(0)
df.tail(n=3)

In [ ]:
#### Plot the data
def plot_result(raw_data, label = "Confirmed Cases", **kwargs):
    
    predicted = kwargs.get('predicted', None)
    
    if predicted is not None:
        interval = len(predicted) // 10
    else:
        interval = len(raw_data) // 10

    day_loc = mdates.DayLocator(interval=interval)  # every day
    date_fmt = mdates.DateFormatter('%Y-%m-%d')

    fig, ax = plt.subplots()
    
    fig.suptitle(label)
    
    ax.scatter(raw_data.index, raw_data.values, label='Actual')
    
    if predicted is not None:
        ax.plot(predicted.index, predicted.values, '--r', label='Fitted')
    

    # format the ticks
    ax.xaxis.set_major_locator(day_loc)
    ax.xaxis.set_major_formatter(date_fmt)
    
    #ax.set_xlim(date_min, date_max)

    # format the coords message box
    ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
    ax.grid(False)

    # rotates and right aligns the x labels, and moves the bottom of the
    # axes up to make room for them
    fig.autofmt_xdate()
    
    ax.legend()
    #plt.legend(['Actual','Fitted'])
        
#### Curve fitting using logistic function
def log_curve(x, k, x_0, ymax):
    return ymax / (1 + np.exp(-k*(x-x_0)))

In [ ]:
## Process the data
df.date = pd.to_datetime(df.date)

ph_total_cases = df[(df.total_cases > 0)].total_cases.to_list()
#ph_date = df[(df.total_cases > 0)].date.dt.strftime('%Y-%m-%d').to_list()
#date_forecast = pd.date_range(, periods=100).to_pydatetime().tolist()
#date_forecast = np.array(pd.date_range("2020-04-10", periods=30).strftime('%Y-%m-%d').tolist())

ph_date = df[(df.total_cases > 0)].date.to_list()
ph_date_forecast = pd.date_range("2020-04-10", periods=30).tolist()


y_ph_total_cases = np.array(ph_total_cases[-30:])
x_ph_total_cases = np.arange(0, len(y_ph_total_cases))

x_ph_date = np.array(ph_date[-30:])
x_ph_date_forecast = np.array(ph_date_forecast)

In [ ]:
## Plot the confirmed cases
confirmed_cases = pd.Series(y_ph_total_cases, index=x_ph_date)
plot_result(confirmed_cases, 'Confirmed Cases')



In [ ]:
#### Fit the curve
params, params_covariance = optimize.curve_fit(log_curve, x_ph_total_cases, y_ph_total_cases, 
                                               bounds=([0,0,0],np.inf))
estimated_k, estimated_x_0, ymax = params

print(params)

In [ ]:
# Plot the fitted curve
k = estimated_k
x_0 = estimated_x_0

x_fitted = np.arange(0, len(y_ph_total_cases) + 30)
y_fitted = log_curve(x_fitted, k, x_0, ymax)

In [ ]:
x_label = np.append(x_ph_date, x_ph_date_forecast, axis=0)

forecast_cases = pd.Series(y_fitted, index=x_label)
plot_result(confirmed_cases, 'Confirmed Cases', predicted=forecast_cases)

In [ ]:
forecast_cases.tail()